# TAREA II - SEP I/A
<div style="text-align: justify;">

**PARTICIPANTES DEL PROYECTO:** NICOLÁS GONZÁLEZ - IVÁN TAPIA

In [ ]:
import numpy as np

def calcular_matriz_admitancia(num_barras, datos_barras):
    # Inicializar matriz de admitancia Y_bus
    Y_bus = np.zeros((num_barras, num_barras), dtype=complex)
    # Procesar cada barra para llenar la matriz Y_bus
    for i in range(num_barras):
        tipo_barra = datos_barras[i]['tipo']
        if tipo_barra == 'slack':
            # Barra slack (barra de referencia)
            Y_bus[i, i] = 1e9  # Valor alto para mantener fijo el voltaje
        else:
            # Calcular admitancia shunt (si es barra PQ o PV)
            Y_shunt = 1 / (datos_barras[i]['R'] + 1j * datos_barras[i]['X'])
            # Llenar diagonal principal de Y_bus con admitancias shunt
            Y_bus[i, i] = Y_shunt
            # Llenar elementos correspondientes a las conexiones entre barras
            for j in range(num_barras):
                if i != j:
                    if tipo_barra == 'PQ' and datos_barras[j]['tipo'] == 'PQ':
                        # Si ambas son PQ
                        Y_bus[i, j] = -1 / (datos_barras[i]['R'] + 1j * datos_barras[i]['X'])
                    elif (tipo_barra == 'PQ' and datos_barras[j]['tipo'] == 'PV') or (tipo_barra == 'PV' and datos_barras[j]['tipo'] == 'PQ'):
                        # Si es PQ-PV o PV-PQ
                        Y_bus[i, j] = 1 / (datos_barras[i]['R'] + 1j * datos_barras[i]['X'])
    print('Matriz Y_bus antes de resolver:')
    print(Y_bus)
    return Y_bus

def calcular_potencia_inyectada(num_barras, datos_barras):
    P_iny = np.zeros(num_barras)
    Q_iny = np.zeros(num_barras)
    
    for i in range(num_barras):
        P_iny[i] = datos_barras[i]['P']
        Q_iny[i] = datos_barras[i]['Q']
    
    return P_iny, Q_iny

def calcular_jacobiano(num_barras, Y_bus, V, theta):
    J = np.zeros((2*num_barras, 2*num_barras))
    
    for i in range(num_barras):
        for j in range(num_barras):
            if i == j and i != 0:  # Excluir la barra slack (barra 0)
                J[i, j] = -Y_bus[i, i].real * V[i]**2 - sum(Y_bus[i, k] * V[i] * V[k] * np.sin(theta[i] - theta[k]) for k in range(num_barras if j != i else 0))
                J[i, j + num_barras] = -sum(Y_bus[i, k] * V[i] * V[k] * np.cos(theta[i] - theta[k]) for k in range(num_barras if j != i else 0))
            elif i != 0 and j != 0:
                J[i, j] = Y_bus[i, j] * V[i] * V[j] * np.sin(theta[i] - theta[j])
                J[i, j + num_barras] = -Y_bus[i, j] * V[i] * V[j] * np.cos(theta[i] - theta[j])
    
    return J

def calcular_flujo_potencia(num_barras, datos_barras, tol=1e-8, max_iter=30):
    # Inicialización de datos
    V = np.ones(num_barras)
    theta = np.zeros(num_barras)
    
    # Definir la barra slack (barra 0)
    V[0] = 1.0  # Voltaje fijo en 1.0 pu
    theta[0] = 0.0  # Ángulo fijo en 0.0 grados
    
    iteracion = 0
    error = tol + 1
    
    # Obtener matriz de admitancia y potencias inyectadas
    Y_bus = calcular_matriz_admitancia(num_barras, datos_barras).astype(complex)
    P_iny, Q_iny = calcular_potencia_inyectada(num_barras, datos_barras)
    
    while error > tol and iteracion < max_iter:
        # Calcular Jacobiano
        J = calcular_jacobiano(num_barras, Y_bus, V, theta)
        
        # Calcular vector de potencias
        F = np.concatenate((np.dot(Y_bus, V) - P_iny, np.dot(Y_bus, V) - Q_iny))
        
        # Excluir la barra slack (barra 0) del vector de potencias
        F[0] = 0.0
        #depuracion
        print('Matriz J:')
        print(J)
        print('Vector F:')
        print(F)
        
        # Calcular incrementos delta(V) y delta(theta) usando el método de Newton-Raphson
        delta = np.linalg.solve(J, -F)
        
        delta_V = delta[:num_barras]
        delta_theta = delta[num_barras:]
        
        # Actualizar V y theta (excluir la barra slack)
        V[1:] += delta_V[1:]
        theta[1:] += delta_theta[1:]
        
        # Calcular error
        error = max(abs(F))
        
        # Incrementar contador de iteraciones
        iteracion += 1
    
    if error <= tol:
        print(f'Convergencia lograda en {iteracion} iteraciones.')
    else:
        print('El método no convergió.')
    
    # Devolver resultados finales (voltajes y ángulos)
    return V, np.degrees(theta)

In [25]:
import numpy as np

#DEFINICIÓN DE BASES Y PARÁMETROS DE LÍNEA - BARRA
R_km = 0.099
X_km = 0.156
Sb_VA = 100E6
Zb = (220E3**2)/Sb_VA
R_pu_km = R_km/Zb
X_pu_km = X_km/Zb 
P_pu = 150E6/Sb_VA
Q_pu = 150E6/Sb_VA

num_barras = 6
datos_barras = [
    {'type': 'PV', 'name': '2', 'P': 0, 'V': 1},                   # Barra 2 (PV)
    {'type': 'PQ', 'name': '1A', 'P': -1*P_pu*0.20, 'Q': -1*Q_pu*0.20},  # Barra 1A (PQ)
    {'type': 'PQ', 'name': '2A', 'P': -1*P_pu*0.35, 'Q': -1*Q_pu*0.35},  # Barra 2A (PQ)
    {'type': 'PQ', 'name': '3A', 'P': -1*P_pu*0.15, 'Q': -1*Q_pu*0.15},  # Barra 3A (PQ)
    {'type': 'PQ', 'name': '2B', 'P': -1*P_pu*0.60, 'Q': -1*Q_pu*0.60},  # Barra 2B (PQ)
    {'type': 'PQ', 'name': '1B', 'P': -1*P_pu*0.10, 'Q': -1*Q_pu*0.10},  # Barra 1B (PQ)
]
datos_lineas =[
    {'Bar_in':'2'  ,'Bar_out':'1A', 'R': R_pu_km*10, 'X': X_pu_km*10}, # Linea 2-1A
    {'Bar_in':'1A' ,'Bar_out':'2A', 'R': R_pu_km*15, 'X': X_pu_km*15}, # Linea 1A-2A
    {'Bar_in':'2A' ,'Bar_out':'3A', 'R': R_pu_km*20, 'X': X_pu_km*20}, # Linea 2A-3A
    {'Bar_in':'3A' ,'Bar_out':'2B', 'R': R_pu_km*15, 'X': X_pu_km*15}, # Linea 3A-2B
    {'Bar_in':'2B' ,'Bar_out':'1B', 'R': R_pu_km*30, 'X': X_pu_km*30}, # Linea 2B-1B
    {'Bar_in':'1B' ,'Bar_out':'2',  'R': R_pu_km*10, 'X': X_pu_km*10}  # Linea 1B-2
]

def calcular_matriz_admitancia(num_barras, datos_barras, datos_lineas):
    # Inicialización de la matriz Y_Bus
    Y_Bus = np.zeros((num_barras, num_barras), dtype=np.complex128)
    # Iterar sobre cada línea para obtener la matriz local y agregar a Y_Bus
    LB = [] #Lista de Nombres de Barras
    for p in range(num_barras): 
        LB.append(datos_barras[p]['name'])
    for linea in datos_lineas:
        Bar_in = linea['Bar_in']
        Bar_out = linea['Bar_out']
        R = linea['R']
        X = linea['X']
        # Calcular la matriz local
        Z_linea = R + 1j*X
        m = np.array([[1/Z_linea, -1/Z_linea], [-1/Z_linea, 1/Z_linea]])
        # Obtener índices de las barras en LB (Lista de Nombres de Barras)
        idx_in = next((i for i, v in enumerate(datos_barras) if v['name'] == Bar_in), None)
        idx_out = next((i for i, v in enumerate(datos_barras) if v['name'] == Bar_out), None)
        # Añadir contribución a Y_Bus
        if idx_in is not None and idx_out is not None:
            Y_Bus[idx_in, idx_in] += m[0, 0]  # Y_in,in += m11
            Y_Bus[idx_in, idx_out] += m[0, 1]  # Y_in,out += m12
            Y_Bus[idx_out, idx_in] += m[1, 0]  # Y_out,in += m21
            Y_Bus[idx_out, idx_out] += m[1, 1]  # Y_out,out += m22
    return Y_Bus

#Muestra Matriz Y_BUS        
BUS = calcular_matriz_admitancia(num_barras,datos_barras,datos_lineas)
filas, columnas = BUS.shape
for fila in BUS:
    for elemento in fila:
        print(f"{elemento.real: 9.4f}{elemento.imag:+9.4f}j", end="\t")
    print()

#def evalua_f(num_barras, datos_barras):



 280.7277-442.3587j	-140.3638+221.1794j	   0.0000  +0.0000j	   0.0000  +0.0000j	   0.0000  +0.0000j	-140.3638+221.1794j	
-140.3638+221.1794j	 233.9397-368.6323j	 -93.5759+147.4529j	   0.0000  +0.0000j	   0.0000  +0.0000j	   0.0000  +0.0000j	
   0.0000  +0.0000j	 -93.5759+147.4529j	 163.7578-258.0426j	 -70.1819+110.5897j	   0.0000  +0.0000j	   0.0000  +0.0000j	
   0.0000  +0.0000j	   0.0000  +0.0000j	 -70.1819+110.5897j	 163.7578-258.0426j	 -93.5759+147.4529j	   0.0000  +0.0000j	
   0.0000  +0.0000j	   0.0000  +0.0000j	   0.0000  +0.0000j	 -93.5759+147.4529j	 140.3638-221.1794j	 -46.7879 +73.7265j	
-140.3638+221.1794j	   0.0000  +0.0000j	   0.0000  +0.0000j	   0.0000  +0.0000j	 -46.7879 +73.7265j	 187.1518-294.9058j	


In [11]:

# Calcular la matriz de admitancia
Y_Bus = calcular_matriz_admitancia(num_barras, datos_barras, datos_lineas)

# Calcular las tensiones de barra con el método de Newton-Raphson
voltajes = newton_raphson(num_barras, datos_barras, Y_Bus)

print("Voltajes de barra:")
for i in range(num_barras):
    print(f"Barra {datos_barras[i]['name']}: {voltajes[i]}")

KeyError: 'Q'

In [7]:
#DEFINICIÓN DE BASES Y PARÁMETROS DE LÍNEA - BARRA
R_km = 0.099
X_km = 0.156
Sb_VA = 100E6
Zb = (220E3**2)/Sb_VA
R_pu_km = R_km/Zb
X_pu_km = X_km/Zb 
P_pu = 150E6/Sb_VA
Q_pu = 150E6/Sb_VA

num_barras = 6
datos_barras = [
    {'type': 'PV', 'name': '2', 'P': 0, 'V': 1},                   # Barra 2 (PV)
    {'type': 'PQ', 'name': '1A', 'P': -1*P_pu*0.20, 'Q': -1*Q_pu*0.20},  # Barra 1A (PQ)
    {'type': 'PQ', 'name': '2A', 'P': -1*P_pu*0.35, 'Q': -1*Q_pu*0.35},  # Barra 2A (PQ)
    {'type': 'PQ', 'name': '3A', 'P': -1*P_pu*0.15, 'Q': -1*Q_pu*0.15},  # Barra 3A (PQ)
    {'type': 'PQ', 'name': '2B', 'P': -1*P_pu*0.60, 'Q': -1*Q_pu*0.60},  # Barra 2B (PQ)
    {'type': 'PQ', 'name': '1B', 'P': -1*P_pu*0.10, 'Q': -1*Q_pu*0.10},  # Barra 1B (PQ)
]
datos_lineas =[
    {'Bar_in':'2'  ,'Bar_out':'1A', 'R': R_pu_km*10, 'X': X_pu_km*10}, # Linea 2-1A
    {'Bar_in':'1A' ,'Bar_out':'2A', 'R': R_pu_km*15, 'X': X_pu_km*15}, # Linea 1A-2A
    {'Bar_in':'2A' ,'Bar_out':'3A', 'R': R_pu_km*20, 'X': X_pu_km*20}, # Linea 2A-3A
    {'Bar_in':'3A' ,'Bar_out':'2B', 'R': R_pu_km*15, 'X': X_pu_km*15}, # Linea 3A-2B
    {'Bar_in':'2B' ,'Bar_out':'1B', 'R': R_pu_km*30, 'X': X_pu_km*30}, # Linea 2B-1B
    {'Bar_in':'1B' ,'Bar_out':'2',  'R': R_pu_km*10, 'X': X_pu_km*10}  # Linea 1B-2
]

calcular_flujo_potencia(num_barras,datos_barras, datos_lineas)

C:\Users\Hp\AppData\Local\Temp\ipykernel_12932\3969515565.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  J[i, j] = Y_bus[i, j] * V[i] * V[j] * np.sin(theta[i] - theta[j])
C:\Users\Hp\AppData\Local\Temp\ipykernel_12932\3969515565.py:86: ComplexWarning: Casting complex values to real discards the imaginary part
  J[i, j + num_barras] = -Y_bus[i, j] * V[i] * V[j] * np.cos(theta[i] - theta[j])


[[   0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.        ]
 [   0.         -233.93971351   -0.            0.            0.
     0.            0.            0.           93.5758854     0.
     0.            0.        ]
 [   0.           -0.         -163.75779946   -0.            0.
     0.            0.           93.5758854     0.           70.18191405
     0.            0.        ]
 [   0.            0.           -0.         -163.75779946   -0.
     0.            0.            0.           70.18191405    0.
    93.5758854     0.        ]
 [   0.            0.            0.           -0.         -140.3638281
    -0.            0.            0.            0.           93.5758854
     0.           46.7879427 ]
 [   0.            0.            0.            0.           -0.
  -187.15177081    0.            0.            0.            0.
    46.7879427     0.        ]
 [   0.            0.   

LinAlgError: Singular matrix